In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 Vertex AI Vizier 和开源 Vizier 之间进行转换

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vizier/conversions_vertex_vizier_and_open_source_vizier.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vizier/conversions_vertex_vizier_and_open_source_vizier.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/vizier/conversions_vertex_vizier_and_open_source_vizier.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

概述

本教程演示了如何在[Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview) 和 [开源(OSS) Vizier](https://oss-vizier.readthedocs.io/)之间迁移代码。 OSS Vizier 是一个基于Python的黑盒优化和研究服务。 它允许您设置一个可以承载黑盒优化算法的OSS Vizier服务器，用于调整目标函数并定义实现新优化算法的抽象和实用程序。

### 目标

在本教程中，您将学习如何使用`Vertex AI Vizier`来优化多目标研究，并将代码转换为OSS Vizier。

目标是__`最小化`__目标指标：
   ```
   y1 = r*sin(theta)
   ```

并同时__`最大化`__目标指标：
   ```
   y2 = r*cos(theta)
   ```

以便您在参数空间上进行评估：

   - __`r`__ 取值范围在 [0,1]，

   - __`theta`__ 取值范围在 [0, pi/2]

### 成本

此教程使用Google Cloud的计费组件：

* Vertex AI

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing)并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装执行这个笔记本所需的包。

In [ ]:
import os

# Configure the environment for the Vertex AI Workbench notebook.
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install google-vizier==0.0.4
! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q

### 重新启动内核

一旦您安装了额外的包，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Restart the kernel after pip installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###GPU运行时

本教程不需要GPU运行时。

###设置您的Google Cloud项目

**无论您使用的是哪个笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您将获得$300的免费信用用于支付计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex AI API，Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter用`!`作为shell命令的前缀运行行，并且用`$`作为Python变量的前缀进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your Google Cloud project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if REGION == "[your-region]":
    REGION = "us-central1"

###验证您的Google Cloud帐户

**如果您正在使用Vertex AI工作台笔记本**，您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，按照提示进行身份验证。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐户**。

3. 在**服务账户名称**字段中输入一个名称，然后点击**创建**。

4. 在**为此服务帐户授予项目访问权限**部分，点击**角色**下拉列表。在筛选框中键入“Vertex AI”，选择**Vertex AI管理员**。在筛选框中键入“存储对象管理员”，选择**存储对象管理员**。

5. 点击*创建*。包含您的密钥的JSON文件将下载到您的本地环境中。

6. 在下面的单元格中将您的服务帐户密钥路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your Google Cloud
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

导入库并定义常量

In [ ]:
import datetime
import math

## 教程

本部分定义了一些参数来创建研究并优化目标函数。

In [ ]:
# These will be automatically filled in.
STUDY_DISPLAY_NAME = "{}_study_{}".format(
    PROJECT_ID.replace("-", ""), datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
)

print("REGION: {}".format(REGION))

### 定义参数

以下是一个示例研究配置，构建为一个层次结构的Python字典。已经填写好。运行该单元格以配置研究。

__`USE_VERTEX_VIZIER`__: 如果为True，则使用Vertex Vizier SDK进行优化。否则使用OSS Vizier。

__`SUGGESTION_COUNT`__: 在一个请求中请求的建议（试验）的数量。

__`MAX_NUM_ITERATIONS`__: 在停止之前要探索的迭代次数。设置为4以缩短运行代码的时间，因此不要期望收敛。要达到收敛，可能需要大约20。

In [ ]:
USE_VERTEX_VIZIER = True  # @param {type:"boolean"}

MAX_NUM_ITERATIONS = 4  # @param {type:"integer"}

SUGGESTION_COUNT = 2  # @param {type:"integer"}

OWNER = "owner"  # @param {type:"string"}

SERVICE_ENDPOINT = "127.0.0.1:8888"  # @param {type:"string"}

### 导入包并为不同来源定义 `create_study`

在 Vertex Vizier 中，`project` 和 `location` 已经指定，并调用 `Study.create_or_load` 来创建一项研究。 您需要输入您研究的所有者和服务器地址的格式 [ip:port]。 要启动 OSS Vizier 服务器，请按照 OSS Vizier 网站上的[说明](https://oss-vizier.readthedocs.io/)进行操作。

In [ ]:
if USE_VERTEX_VIZIER:
    from google.cloud import aiplatform
    from google.cloud.aiplatform.vizier import Study, pyvizier

    def create_study(project, location, display_name, problem):
        aiplatform.init(project=project, location=location)
        study = Study.create_or_load(display_name=display_name, problem=problem)
        return study

else:
    from vizier.service import clients, pyvizier

    def create_study(project, location, display_name, problem):
        clients.environment_variables.service_endpoint = SERVICE_ENDPOINT
        study = clients.Study.from_study_config(
            problem, owner=OWNER, study_id=STUDY_DISPLAY_NAME
        )
        return study

### 度量评估函数

接下来，定义一些函数来评估这两个客观指标。

In [ ]:
# r * sin(theta)
def Metric1Evaluation(r, theta):
    """Evaluate the first metric on the trial."""
    return r * math.sin(theta)


# r * cos(theta)
def Metric2Evaluation(r, theta):
    """Evaluate the second metric on the trial."""
    return r * math.cos(theta)


def CreateMetrics(r, theta):
    # Evaluate both objective metrics for this trial
    y1 = Metric1Evaluation(r, theta)
    y2 = Metric2Evaluation(r, theta)
    print(
        "[r = {}, theta = {}] => y1 = r*sin(theta) = {}, y2 = r*cos(theta) = {}".format(
            r, theta, y1, y2
        )
    )
    measurement = pyvizier.Measurement()
    measurement.metrics["y1"] = y1
    measurement.metrics["y2"] = y2

    # Return the results for this trial
    return measurement

### 优化

以下代码定义了一个带有参数和指标的研究，根据Vizier的建议评估指标信息，并将指标值报告回来。经过几轮迭代，您可以通过调用 `optimal_trials()` 来获得最佳试验。该代码适用于 Vertex Vizier 和 OSS Vizier。

In [ ]:
problem = pyvizier.StudyConfig()
problem.algorithm = pyvizier.Algorithm.RANDOM_SEARCH

# Objective Metrics
problem.metric_information.append(
    pyvizier.MetricInformation(name="y1", goal=pyvizier.ObjectiveMetricGoal.MINIMIZE)
)
problem.metric_information.append(
    pyvizier.MetricInformation(name="y2", goal=pyvizier.ObjectiveMetricGoal.MAXIMIZE)
)

# Defines the parameters configuration.
root = problem.search_space.select_root()
root.add_float_param("r", 0, 1.0, scale_type=pyvizier.ScaleType.LINEAR)
root.add_float_param("theta", 0, 1.57, scale_type=pyvizier.ScaleType.LINEAR)

study = create_study(
    project=PROJECT_ID,
    location=REGION,
    display_name=STUDY_DISPLAY_NAME,
    problem=problem,
)

for _ in range(MAX_NUM_ITERATIONS):
    trials = study.suggest(count=SUGGESTION_COUNT)
    for trial in trials:
        materialize_trial = trial.materialize()
        measurement = CreateMetrics(
            materialize_trial.parameters.get_value("r"),
            materialize_trial.parameters.get_value("theta"),
        )
        trial.add_measurement(measurement=measurement)
        trial.complete(measurement=measurement)

optimal_trials = study.optimal_trials()
print("optimal_trials: {}".format(optimal_trials))

清理工作

要清理这个项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。您还可以通过运行以下代码手动删除创建的资源。

In [ ]:
study.delete()